In [ ]:
import os

from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import numpy as np

from src.coordinates import sph_to_cart, cyl_to_cart
from src.normals import sph_normals, cyl_normals
from src.plotting import set_defense_context, set_axes_equal

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# constants

elev, azim = 20, -60  # 3-D view
N = 7
edge_length = 2  # in cm
A = edge_length ** 2  # in cm2
target_area_origin = (-edge_length/2, -edge_length/2)
r = 2  # in cm
alpha = 2 * np.arcsin(edge_length/2/r)  # compute angle from secant
rectangle_kwargs = dict(width=edge_length, height=edge_length,
                        ec='k', fc='gray', lw=2, alpha=0.25)
scatter_kwargs = dict(s=10, c='k', depthshade=True)
quiver_kwargs = dict(normalize=True, lw=0.5, length=0.75, color='k')

In [ ]:
def add_coordinate_frame(ax):
    """Set RGB coordinate frame to axes.
    
    Parameters
    ----------
    ax : matplotlib.axes._subplots.Axes3DSubplot
        3-D axes subplot.
    
    Returns
    -------
    matplotlib.axes._subplots.Axes3DSubplot
        Axes with coordinate frame.
    """
    ax.quiver(-1.5, -1, -1.5, 0.75, 0, 0, color='r')
    ax.text(-0.7, -1, -1.5, s='$x$', color='r', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0.75, 0, color='g')
    ax.text(-1.5, -0.2, -1.5, s='$y$', color='g', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0, 0.75, color='b')
    ax.text(-1.5, -1, -0.7, s='$z$', color='b', fontweight='bold')
    ax.scatter(-1.5, -1, -1.5, s=10, color='k', depthshade=False)
    return ax

# Planar evaluation surface

In [ ]:
# point cloud

x = np.linspace(-edge_length/2, edge_length/2, N)
y = 0.0
z = np.linspace(-edge_length/2, edge_length/2, N)
Xt, Zt = np.meshgrid(x, z)
xt_pln = Xt.ravel()
yt_pln = np.zeros_like(xt_pln)
zt_pln = Zt.ravel()

In [ ]:
# normals

nx_pln = 0
ny_pln = -1
nz_pln = 0

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    plane = Rectangle(target_area_origin, **rectangle_kwargs)
    ax.add_patch(plane)
    pathpatch_2d_to_3d(plane, z=y, zdir='y')
    ax.scatter(xt_pln, yt_pln, zt_pln, **scatter_kwargs)
    ax.quiver(xt_pln, yt_pln, zt_pln,
              nx_pln, ny_pln, nz_pln,
              **quiver_kwargs)
    ax = add_coordinate_frame(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'planar.png'
    fig.savefig(os.path.join('figures', 'evaluation-surface', fname),
                dpi=500, bbox_inches='tight')

# Non-planar evaluation surface

## Spherical evaluation surface

In [ ]:
# point cloud

theta = np.linspace(np.pi/2 - alpha/2, np.pi/2 + alpha/2, N)
phi = np.linspace(np.pi-alpha/2, np.pi+alpha/2, N)
Theta, Phi = np.meshgrid(theta, phi)
yt_sph, xt_sph, zt_sph = sph_to_cart(r, Theta.ravel(), Phi.ravel())
yt_sph -= yt_sph.min()

In [ ]:
# compute normals

ny_sph, nx_sph, nz_sph = sph_normals(r, Theta.ravel(), Phi.ravel())

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    plane = Rectangle(target_area_origin, **rectangle_kwargs)
    ax.add_patch(plane)
    pathpatch_2d_to_3d(plane, z=y, zdir='y')
    ax.scatter(xt_sph, yt_sph, zt_sph,
               **scatter_kwargs)
    ax.quiver(xt_sph, yt_sph, zt_sph,
              nx_sph, ny_sph, nz_sph,
              **quiver_kwargs)
    ax = add_coordinate_frame(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'spherical.png'
    fig.savefig(os.path.join('figures', 'evaluation-surface', fname),
                dpi=500, bbox_inches='tight')

## Cylindrical evaluation surface

In [ ]:
# point cloud

theta = np.linspace(np.pi/2-alpha/2, np.pi/2+alpha/2, N)
Theta, Zt = np.meshgrid(-theta, z)
xt_cyl, yt_cyl, zt_cyl = cyl_to_cart(r, Theta.ravel(), Zt.ravel())
yt_cyl -= yt_cyl.min()

In [ ]:
# compute normals

nx_cyl, ny_cyl, nz_cyl = cyl_normals(r, Theta.ravel(), zt_cyl)

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    plane = Rectangle(target_area_origin, **rectangle_kwargs)
    ax.add_patch(plane)
    pathpatch_2d_to_3d(plane, z=y, zdir='y')
    ax.scatter(xt_cyl, yt_cyl, zt_cyl,
               **scatter_kwargs)
    ax.quiver(xt_cyl, yt_cyl, zt_cyl,
              nx_cyl, ny_cyl, nz_cyl,
              **quiver_kwargs)
    ax = add_coordinate_frame(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'cylindrical.png'
    fig.savefig(os.path.join('figures', 'evaluation-surface', fname),
                dpi=500, bbox_inches='tight')